**Set up modules/data**

In [6]:
import numpy as np
import pandas as pd
import math
pd.options.mode.copy_on_write = True
import random
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)

Load data set

In [7]:
UCL = pd.read_csv('../Data/key_stats.csv')

UCL
#we can add other stats to make mega data frame

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,0,64.2
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,6,133.0
2,Benzema,Real Madrid,Forward,1106,12,15,1,121.5
3,Modrić,Real Madrid,Midfielder,1077,13,0,4,124.5
4,Éder Militão,Real Madrid,Defender,1076,12,0,0,110.4
...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0,0.7
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0,0.7
744,Cojocari,Sheriff,Defender,1,1,0,0,0.5
745,Maouassa,Club Brugge,Defender,1,1,0,0,0.2


In [ ]:
#example, combines key stats/disiplinary

test_CL = pd.read_csv('../Data/disciplinary.csv')
merged_df = pd.merge(UCL, test_CL, on="player_name", how="outer")  # outer keeps all players from both files

merged_df = merged_df.fillna(0)
merged_df

,player_name,club_x,position_x,minutes_played_x,match_played_x,goals,assists,distance_covered,serial,club_y,position_y,fouls_committed,fouls_suffered,red,yellow,minutes_played_y,match_played_y
0,Aaronson,Salzburg,Midfielder,715,8,0,2,103.5,326.0,Salzburg,Midfielder,3.0,13.0,0.0,0.0,715.0,8.0
1,Abubakari,Malmö,Forward,116,4,0,0,15.3,146.0,Malmö,Forward,6.0,4.0,0.0,0.0,116.0,4.0
2,Acuña,Sevilla,Defender,379,5,0,0,43.2,31.0,Sevilla,Defender,12.0,6.0,0.0,0.0,379.0,5.0
3,Adams,Leipzig,Midfielder,292,5,0,0,38.9,146.0,Leipzig,Midfielder,6.0,2.0,4.0,0.0,292.0,5.0
4,Adamu,Salzburg,Forward,231,8,1,0,32.6,55.0,Salzburg,Forward,10.0,2.0,0.0,0.0,231.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,Óscar Rodríguez,Sevilla,Midfielder,22,1,0,0,3.8,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
764,Čolak,Malmö,Forward,500,6,0,0,63,40.0,Malmö,Forward,11.0,7.0,1.0,0.0,500.0,6.0
765,Šeško,Salzburg,Forward,234,6,0,0,32.1,263.0,Salzburg,Forward,4.0,0.0,0.0,0.0,234.0,6.0
766,Šimić,Salzburg,Forward,4,1,0,0,1.3,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


**Logistic Regression:**

Find stastically significant features in data set

Run model to see what gives test data

Run k-fold to test data since less than 1000 observations

**K-means Clustering**

Normalize data

Run k-means, we can discuss what best one is or switch to hierarchial

In [ ]:
#I was thinking 4 for 4 positions or we can research how many football archetypes there are

**Random Forest**

Run the random forest

Compare to logistic regression